In [ ]:
#  Copyright private in 2018 
#  Modify Date: 
#      2018-9-30 
#  Purpose : 
#      1. Jieba 分词
#      2. Remove stopwords
#      3. LDA 模型  
#      4. TF-IDF 关键字提取
#      5. CountVectorizer/TfidfVectorizer 构造词向量 + NavieBayesClassifier  
# ----------
#coding:utf-8


In [203]:
import pandas as pd
import numpy as np
import jieba

In [274]:
# reading news data from local csv file

In [275]:
df_news =  pd.read_csv('./data/news.csv',names = ['url','contenttitle','content','category'],encoding = 'utf-8')
df_news =  df_news.dropna()
df_news.head()

,url,contenttitle,content,category
0,http://news.sohu.com/20120612/n345428229.shtml,公安机关销毁１０余万非法枪支 跨国武器走私渐起,中广网唐山６月１２日消息（记者汤一亮 庄胜春）据中国之声《新闻晚高峰》报道，今天（１２日）上...,社会
1,http://news.sohu.com/20120607/n344998325.shtml,张绍刚发道歉信网友不认可：他的问题是俯视他人（图）,天津卫视求职节目《非你莫属》“晕倒门”事件余波未了，主持人张绍刚前日通过《非你莫属》节目组发...,娱乐
2,http://news.sohu.com/20120604/n344745879.shtml,＃（关注夏收）（３）夫妻“麦客”忙麦收,临沂（山东），２０１２年６月４日 夫妻“麦客”忙麦收 ６月４日，在山东省临沂市郯城县郯城街道...,农业
3,http://news.sohu.com/20120613/n345535702.shtml,欧洲杯大战在即 荷兰葡萄牙面临淘汰将背水一战,中广网北京６月１３日消息（记者王宇）据中国之声《新闻晚高峰》报道，明天凌晨两场欧洲杯的精彩比...,体育
4,http://news.sohu.com/20120601/n344598651.shtml,扎克伯格携妻罗马当街吃３０元麦当劳午餐（组图）,环球网记者李亮报道，正在意大利度蜜月的“脸谱”创始人扎克伯格与他华裔妻子的一举一动都处于媒体...,科技


In [206]:
#df_news.shape()

In [207]:
# content show
content  = df_news.content.values.tolist()
category = df_news.category.values.tolist()

# print content and category
print(content[3])
print(category[3])

中广网北京６月１３日消息（记者王宇）据中国之声《新闻晚高峰》报道，明天凌晨两场欧洲杯的精彩比赛上演，死亡之组Ｂ组当中两支传统的强队荷兰队和葡萄牙队正面临着提前淘汰出局的危险。第一轮之后Ｂ组的积分的形势是３、３、０、０，德国和丹麦３分，葡萄牙和荷兰０分。今天又是３分的两支球队分别对阵０分的两支球队，如果０分的再输就会被淘汰，而３分的如果再赢就可以提前出线。换句话说，德国和丹麦今天晚上赢球就能出线，葡萄牙和荷兰如果输球就要回家。葡萄牙：Ｃ罗输球后失声痛哭北京时间今天晚上１２点丹麦对葡萄牙的比赛将在乌克兰的利沃夫进行，明天凌晨２点４５分德国对荷兰的比赛将在乌克兰的哈尔科夫进行。在输球就要回家的阴影笼罩之下，荷兰和葡萄牙队身上的压力可想而知。第一轮比赛中葡萄牙队负于德国。不过德国的实力强于葡萄牙强，葡萄牙队受到的质疑并不算大，但是头号球星罗纳尔多对自己的要求非常严格，现在全队的压力有七成都集中在了罗纳尔多身上，他想在国家队证明自己的欲望也非常强烈。虽然据说输球之后他甚至失声痛哭，但他后来表示在与皇马的教练穆里尼奥谈过之后，他相信葡萄牙队一定能够赢得这场球，带领球队走出困境。荷兰：媒体曝球队＂内讧＂和葡萄牙队相比，荷兰队在第一场输球以后产生的负能量更多一些。早在和丹麦队比赛之前就有媒体传出荷兰队出现了内讧，核心的人物就是范佩西和亨特拉尔这两名主力前锋的竞争者。而输给丹麦之后矛盾越来越多，涉及到的人也越来越多，有球员和球员之间的，也有球员和教练之间的。亨特拉尔对自己不能首发上场不满意，范德法特也抵触替补的角色，中场的德容质疑主帅是任人唯亲，在第一场里边把自己提前换下去没有把自己的女婿范博梅尔提前换下去。对于这样种种的矛盾，中场大将斯内德表示队中确实有人太过于自我，承认目前球队的气氛不如两年前，但是他也认为输掉第一场比赛之后出现这样的情况是正常的，但是不管怎么说如果荷兰想要战胜强大的德国队，必须先整理好内部。对于球迷来说，今天晚上的比赛会非常有吸引力，在荷兰和葡萄牙必须赢球的情况下，死亡之组的比赛将不会像首轮那样沉闷，一定会精彩纷呈。
体育


In [208]:
## category
category = df_news.category.values.tolist()
print(category[3])

体育


##  1. 分词: 使用jieba分词器

In [278]:
# 去掉换行和回车符  - 
content_S = []
# line 表示当前新闻的一条主体
for line in content: 
    current_segment =  jieba.lcut(line)
    if len(current_segment) >1 and current_segment != '\r\n': # 不包括回车或者换行符
        content_S.append(current_segment)

In [279]:
content_S[3]

['中广网',
 '北京',
 '６',
 '月',
 '１',
 '３',
 '日',
 '消息',
 '（',
 '记者',
 '王宇',
 '）',
 '据',
 '中国',
 '之声',
 '《',
 '新闻',
 '晚',
 '高峰',
 '》',
 '报道',
 '，',
 '明天',
 '凌晨',
 '两场',
 '欧洲杯',
 '的',
 '精彩',
 '比赛',
 '上演',
 '，',
 '死亡',
 '之组',
 'Ｂ',
 '组',
 '当中',
 '两支',
 '传统',
 '的',
 '强队',
 '荷兰队',
 '和',
 '葡萄牙队',
 '正',
 '面临',
 '着',
 '提前',
 '淘汰',
 '出局',
 '的',
 '危险',
 '。',
 '\ue40c',
 '第一轮',
 '之后',
 'Ｂ',
 '组',
 '的',
 '积分',
 '的',
 '形势',
 '是',
 '３',
 '、',
 '３',
 '、',
 '０',
 '、',
 '０',
 '，',
 '德国',
 '和',
 '丹麦',
 '３',
 '分',
 '，',
 '葡萄牙',
 '和',
 '荷兰',
 '０',
 '分',
 '。',
 '今天',
 '又',
 '是',
 '３',
 '分',
 '的',
 '两支',
 '球队',
 '分别',
 '对阵',
 '０',
 '分',
 '的',
 '两支',
 '球队',
 '，',
 '如果',
 '０',
 '分',
 '的',
 '再',
 '输',
 '就',
 '会',
 '被',
 '淘汰',
 '，',
 '而',
 '３',
 '分',
 '的',
 '如果',
 '再赢',
 '就',
 '可以',
 '提前',
 '出线',
 '。',
 '换句话说',
 '，',
 '德国',
 '和',
 '丹麦',
 '今天',
 '晚上',
 '赢球',
 '就',
 '能',
 '出线',
 '，',
 '葡萄牙',
 '和',
 '荷兰',
 '如果',
 '输球',
 '就要',
 '回家',
 '。',
 '\ue40c',
 '葡萄牙',
 '：',
 'Ｃ',
 '罗',
 '输球',
 '后',
 '失声痛哭',
 '\ue40c',
 '北京',
 '时

In [211]:
# 当前的结果
pd_content  = pd.DataFrame( {'content_S':content_S} )
pd_content.head()

,content_S
0,"[中广网, 唐山, ６, 月, １, ２, 日, 消息, （, 记者, 汤一亮, , 庄胜..."
1,"[天津, 卫视, 求职, 节目, 《, 非你莫属, 》, “, 晕倒, 门, ”, 事件, ..."
2,"[临沂, （, 山东, ）, ，, ２, ０, １, ２, 年, ６, 月, ４, 日, ..."
3,"[中广网, 北京, ６, 月, １, ３, 日, 消息, （, 记者, 王宇, ）, 据, ..."
4,"[环球网, 记者, 李亮, 报道, ，, 正在, 意大利, 度蜜月, 的, “, 脸谱, ”..."


## 2. 去掉stopswords

In [277]:
# Stopwords Show 

stopwords = pd.read_csv("./data/stopwords.txt",index_col = False, sep = "\t",quoting = 3, names = ['stopwords'],encoding= 'utf-8')
stopwords.head()

,stopwords
0,"2010,?、。“”《》！，：；？.........。。。。。。。。。123457890下一..."


In [213]:
# remove stop words functions 
def remove_stopwords(contents,stopwords):
    
    content_clean = []  # content that removed stopward 
    all_words     = []  # all word removed stopwords
    for line in contents:
        line_clean = []  # every line that removed stopwords 
         
        for word in line:
            if word in stopwords:
                continue             # 如果 word 出现stop words 里面， 则继续执行。跳过
            line_clean.append(word)  # 否则认为是好的词加到line_clean 中
            all_words.append(str(word))
        content_clean.append(line_clean)
    return all_words,content_clean

In [214]:
# call remove_stopwords to remove stop words

contents  = pd_content.content_S.values.tolist()
#stopwords = stopwords.stopwords.values.tolist()
all_words,content_clean = remove_stopwords(contents,stopwords)

In [215]:
# content that remove stopwords
pd_content  = pd.DataFrame({'content_clean':content_clean})
pd_content.head()

,content_clean
0,"[中广网, 唐山, ６, 月, １, ２, 日, 消息, （, 记者, 汤一亮, , 庄胜..."
1,"[天津, 卫视, 求职, 节目, 《, 非你莫属, 》, “, 晕倒, 门, ”, 事件, ..."
2,"[临沂, （, 山东, ）, ，, ２, ０, １, ２, 年, ６, 月, ４, 日, ..."
3,"[中广网, 北京, ６, 月, １, ３, 日, 消息, （, 记者, 王宇, ）, 据, ..."
4,"[环球网, 记者, 李亮, 报道, ，, 正在, 意大利, 度蜜月, 的, “, 脸谱, ”..."


In [216]:
# all_words 
df_all_words  = pd.DataFrame({'all_words':all_words})
df_all_words.head()

,all_words
0,中广网
1,唐山
2,６
3,月
4,１


In [217]:
import numpy
words_count = df_all_words.groupby(by = ['all_words'])['all_words'].agg({"count":numpy.size})

# 按照值进行排序 - reset_index
words_count =words_count.reset_index().sort_values(by = ["count"],ascending = False)

words_count.head()

c:\users\h155809\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  


,all_words,count
1668,，,327
1173,的,183
8,。,128
7,、,80
1671,１,68


## 3.LDA 主题模型

### 3.1 TF-IDF 关键字提取

In [218]:
#df_news =  pd.read_csv('C:/Python_/data/news.csv',names = ['url','contenttitle','content'],encoding = 'utf-8')
#df_news =  df_news.dropna()
#df_news.head()

import jieba.analyse
index = 4
print (df_news['content'][index])
content_S_str = "".join(content_S[index])

print("关键字：")
print (" ".join(jieba.analyse.extract_tags(content_S_str,topK = 4, withWeight = False)))

环球网记者李亮报道，正在意大利度蜜月的“脸谱”创始人扎克伯格与他华裔妻子的一举一动都处于媒体的追踪之下。５月３１日，在罗马这个拥有１３家米其林星级餐厅的城市，身家近千亿的扎克伯格夫妇被发现穿着普通的Ｔ恤短袖，在路边一家麦当劳花３英镑买了两人的午餐，并坐在街边的台阶上大嚼起来。英国《每日电讯报》５月３１日报道，扎克伯格夫妇的这顿午饭包括汉堡、炸鸡，仅３英镑（约为人民币３０元）。两人先是依偎着看餐牌，然后由妻子点餐，扎克伯格则调皮地拿起手机，给妻子拍照。拎着装满快餐食品的纸袋子，这对明星夫妇竟坐在街边的一个台阶上，边聊天边大吃起来。一位路人告诉记者，“没人认出他们俩。他们只是混在人群中。”延伸阅读：网友在微博上热传，ＣＣＴＶ最近播出的纪录片《中国警察》中，一位疑似脸谱网站创始人扎克伯格的群众演员成最大牌的“路人甲”。据报道，《中国警察》第四集第３１秒左右，出现貌似扎克伯格及其妻子今年３月在上海游玩时候的镜头。对着镜头，“扎克伯格”似乎笑得很开心，走路一甩一甩的让网友大呼喜感。［详细］（中国新闻网）
关键字：
扎克 伯格 妻子 夫妇


### 3.2 LDA 主题模型
#### 主题中有哪些关键词，以及关键词出现的权重

In [219]:
## 格式： list of list  分词好的整个语料
## 一篇文章当中，有哪些,主题中哪些关键词

In [220]:
from gensim import corpora,models,similarities ## 语料库。模型.相似度
import gensim

In [221]:
## 做映射，要提前分好词
dictionary  = corpora.Dictionary(content_clean) # 将词语映射成模型
corpus  =[dictionary.doc2bow(sentence) for sentence in content_clean ]

###Lda 
lda = gensim.models.ldamodel.LdaModel(corpus = corpus, id2word = dictionary , num_topics = 12) # num_topics 多少个主题

In [222]:
## 11 号分类结果
print (lda.print_topic(11,topn = 2))

0.042*"，" + 0.017*"的"


In [223]:
##print all kinds of toptic 
for topic in lda.print_topics(num_topics = 12, num_words =2):
    print (topic[1])

# Issue 
print("明显stopwords 没有work")

0.062*"，" + 0.027*"的"
0.009*"，" + 0.009*"”"
0.030*"，" + 0.020*"的"
0.027*"的" + 0.024*"，"
0.050*"，" + 0.038*"的"
0.052*"，" + 0.029*"的"
0.045*"，" + 0.028*"的"
0.032*"，" + 0.016*"的"
0.040*"，" + 0.030*"的"
0.039*"，" + 0.019*"１"
0.075*"，" + 0.037*"的"
0.042*"，" + 0.017*"的"
明显stopwords 没有work


### 3.2 新闻文本分类任务


In [117]:
df_train  =  pd.DataFrame({'content_clean':content_clean, 'label':df_news['category']})
#df_train.head()
df_train

,content_clean,label
0,"[中广网, 唐山, ６, 月, １, ２, 日, 消息, （, 记者, 汤一亮, , 庄胜...",社会
1,"[天津, 卫视, 求职, 节目, 《, 非你莫属, 》, “, 晕倒, 门, ”, 事件, ...",娱乐
2,"[临沂, （, 山东, ）, ，, ２, ０, １, ２, 年, ６, 月, ４, 日, ...",农业
3,"[中广网, 北京, ６, 月, １, ３, 日, 消息, （, 记者, 王宇, ）, 据, ...",体育
4,"[环球网, 记者, 李亮, 报道, ，, 正在, 意大利, 度蜜月, 的, “, 脸谱, ”...",科技
5,"[本报记者, , 张忠德, , 本报, 通讯员, , 张艳, , 苏婧, , 城区...",娱乐
6,"[中新网, ６, 月, ８, 日电, , 据, 朝中社, 报道, ，, ２, ６, ０, ...",国际
7,"[中新社, 北京, ６, 月, ２, 日电, （, 记者, , 刘辰瑶, ）, 国资委, ...",民生
8,"[证监会, 近日, 召开, 新闻, 通气会, ，, 就, 《, 关于, 加强, 与, 上市公...",经济
9,"[中国台湾, 网, ６, 月, １, ５, 日, 消息, , 据, 台湾, 《, 中国时报...",文化


### 1.所有的标签label

In [114]:
# 所有的label
df_train.label.unique()

array(['社会', '娱乐', '农业', '体育', '科技', '国际', '民生', '经济', '文化', '金融', '政治'],
      dtype=object)

In [115]:
### 将label 值全部替换成数字，因为计算机只能读懂数字

In [116]:
label_mapping = {"社会":1,"娱乐":2,"农业":3,"体育":4,"科技":5, "国际":6, "民生":7,"经济":8,"文化":9,"金融":10,"政治":11}

df_train['label'] = df_train['label'].map(label_mapping)
df_train.head()


,Content_Clean,label
0,"[中广网, 唐山, ６, 月, １, ２, 日, 消息, （, 记者, 汤一亮, , 庄胜...",1
1,"[天津, 卫视, 求职, 节目, 《, 非你莫属, 》, “, 晕倒, 门, ”, 事件, ...",2
2,"[临沂, （, 山东, ）, ，, ２, ０, １, ２, 年, ６, 月, ４, 日, ...",3
3,"[中广网, 北京, ６, 月, １, ３, 日, 消息, （, 记者, 王宇, ）, 据, ...",4
4,"[环球网, 记者, 李亮, 报道, ，, 正在, 意大利, 度蜜月, 的, “, 脸谱, ”...",5


In [118]:
# train _test _split 
# content as the trian data, label as the standard values of prediction 
from sklearn.model_selection import train_test_split 

x_train,x_test,y_train,y_test  = train_test_split(df_train['content_clean'].values,df_train['label'].values )

In [226]:
#x_train.shape

In [227]:
#type(x_train)

In [228]:
#x_train[0][1]

In [243]:
## list to string  for data
def word_tostring(data):
    
    words_train = []
    for line_index in range(len(data)):
        try: # 记录程序执行的异常
            words_train.append(' '.join(x_train[line_index]))  #  将array中的文字，按照空格连接成字符串
        except:
                print ( "exception warning :", line_index, word_index)
    return words_train

In [202]:
#x_train[0]

In [224]:
#words_train[0]

In [225]:
#print(len(words_train))

In [175]:
# Domo -show to constrct vector 
from sklearn.feature_extraction.text import CountVectorizer 
texts = ["dog fish cat","dog cat cat "," fish bird","bird"]

cv     = CountVectorizer(ngram_range = (1,4)) # n garm = 1,2,3
cv_fit = cv.fit_transform(texts)

print(cv.get_feature_names()) 

print(cv_fit.toarray()) # 按照上述打印的类型，将texts中的字符按照数量转换成向量

['bird', 'cat', 'cat cat', 'dog', 'dog cat', 'dog cat cat', 'dog fish', 'dog fish cat', 'fish', 'fish bird', 'fish cat']
[[0 1 0 1 0 0 1 1 1 0 1]
 [0 2 1 1 1 1 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 1 1 0]
 [1 0 0 0 0 0 0 0 0 0 0]]


In [147]:
print( cv_fit.toarray().sum(axis = 0) )  # 0 - 按照列求和。 1——按照行数求和

# bird = 2, cat = 3, dog = 2, fish  = 2 

[2 3 2 2]


In [255]:
### 按照词来构造向量
from sklearn.feature_extraction.text import CountVectorizer 
vec = CountVectorizer(analyzer = 'word', max_features = 10, lowercase = False)
words_train = word_tostring(x_train)
vec.fit(words_train)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=False, max_df=1.0, max_features=10, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [256]:
type(words_train)

list

In [257]:
type(vec.transform(words_train))

scipy.sparse.csr.csr_matrix

### 2. navie 贝叶斯分类器

In [258]:
### 构成 Navie 贝叶斯分类器
from  sklearn.naive_bayes import MultinomialNB
classifier  = MultinomialNB()
classifier.fit(vec.transform(words_train),y_train)  # vec.transform(words) 转words 为向量

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [262]:
classifier.score(vec.transform(words_train),y_train)

0.5555555555555556

In [266]:
# test data train to join to string
words_test = word_tostring(x_test)

In [267]:
words_test[0]

'中国台湾 网 ６ 月 １ ５ 日 消息 \u3000 据 台湾 《 中国时报 》 报道 ， 岛内 大学教授 “ 假发票 、 真 Ａ 钱 ” 案 ， 如 滚雪球 ， 愈演愈烈 ， 目前 共有 台北 、 台 中 、 彰化 、 台南 等 四个 地检署 在 侦办 。 最新 一期 的 《 时报周刊 》 报道 ， 涉案 教授 可能 多达 １ ０ ０ ０ 人 ， 其中 理工 领域 占 三分之二 强 ； 由于 台湾 “ 检察 总长 ” 黄世铭 定调 以 贪污罪 嫌 究办 ， 涉案者 将 面临 牢狱之灾 ， 岛内 各 顶尖 大学 或 学生 ， 都 会 产生 不小 冲击 。 （ 中国台湾 网 \u3000 刘海 伟 ）'

In [273]:
#classifier.score(vec.transform(words_test),y_test)

### 3.TF - IDF + Naive 贝叶斯分类器

In [268]:
# 构造向量
from sklearn.feature_extraction.text import TfidfVectorizer 
Tfidfclassifier = TfidfVectorizer(analyzer = 'word', max_features = 10, lowercase = False)
Tfidfclassifier.fit(words_train)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=False, max_df=1.0, max_features=10, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [269]:
# 构造分类器
from sklearn.naive_bayes import MultinomialNB
classifier  = MultinomialNB()
classifier.fit(vec.transform(words_train),y_train)  # vec.transform(words) 转words 为向量

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [272]:
classifier.score(vec.transform(words_train),y_train) 

0.5555555555555556